
# STX/BTC Macro Correlation Analysis

Automatically generated notebook for exploring macroeconomic relationships with the STX/BTC price ratio.


In [ ]:
HISTORY_DAYS = 730
FORCE_REFRESH = False

In [ ]:

from datetime import datetime, timedelta, timezone

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.config import OUT_DIR
from src.prices import load_price_panel
from src.macro_data import load_macro_panel

END_TS = datetime.now(timezone.utc)
START_TS = END_TS - timedelta(days=HISTORY_DAYS)
START_DATE = START_TS.date().isoformat()
END_DATE = END_TS.date().isoformat()

print(f"Analysis period: {START_DATE} to {END_DATE}")


In [ ]:
print("Fetching STX/BTC price data...")
price_panel = load_price_panel(
    start=START_TS,
    end=END_TS,
    frequency="1D",
    force_refresh=FORCE_REFRESH,
)

print(f"Loaded {len(price_panel)} daily price records")
print(f"Columns: {list(price_panel.columns)}")
price_panel.head()

In [ ]:

print("Fetching macroeconomic indicators...")
macro_panel = load_macro_panel(
    start_date=START_DATE,
    end_date=END_DATE,
    force_refresh=FORCE_REFRESH,
)

print(f"Loaded {len(macro_panel)} macro indicator records")
print(f"Columns: {list(macro_panel.columns)}")
macro_panel.head()


In [ ]:

print("Merging price and macro panels...")

price_panel = price_panel.copy()
price_panel["date"] = pd.to_datetime(price_panel["ts"], utc=True).dt.date
macro_panel = macro_panel.copy()
macro_panel["date"] = pd.to_datetime(macro_panel["date"]).dt.date

full_panel = price_panel.merge(macro_panel, on="date", how="left")

print(f"Merged panel: {len(full_panel)} records")
print("Missing values per column:")
print(full_panel.isnull().sum())

full_panel = full_panel.ffill()
full_panel.head()


In [ ]:
print("Computing correlations with STX/BTC ratio...")

correlation_cols = [
    "sp500_close",
    "sp500_pct_change",
    "unemployment_rate",
    "fed_funds_rate",
    "treasury_10y",
    "vix_close",
    "usdt_supply",
    "usdc_supply",
    "usdt_daily_change",
    "usdc_daily_change",
    "dxy_close",
    "gold_close",
]

available_cols = [
    col for col in correlation_cols if col in full_panel and full_panel[col].notna().any()
]

if available_cols:
    correlations = full_panel[["stx_btc"] + available_cols].corr()["stx_btc"].drop("stx_btc")
    correlations = correlations.sort_values(ascending=False)
    print("\nCorrelations with STX/BTC ratio:")
    print(correlations)

    fig = go.Figure(
        data=go.Heatmap(
            z=[correlations.values],
            x=correlations.index,
            y=["STX/BTC"],
            colorscale="RdBu",
            zmid=0,
            text=[correlations.values],
            texttemplate="%{text:.3f}",
            textfont={"size": 10},
        )
    )
    fig.update_layout(title="Correlation with STX/BTC Ratio", xaxis_title="Indicator", height=300)
    fig.show()
else:
    print("No macro columns available for correlation analysis.")

In [ ]:

print("Creating time series dashboard...")

fig = make_subplots(
    rows=5,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=[
        "STX/BTC Price Ratio",
        "S&P 500 Index",
        "Stablecoin Supply (Tether + USDC)",
        "Interest Rates (Fed Funds & 10Y Treasury)",
        "VIX Volatility Index",
    ],
    specs=[[{"secondary_y": False}],
           [{"secondary_y": False}],
           [{"secondary_y": True}],
           [{"secondary_y": False}],
           [{"secondary_y": False}]],
)

fig.add_trace(
    go.Scatter(x=full_panel["date"], y=full_panel["stx_btc"], name="STX/BTC", line=dict(color="purple")),
    row=1,
    col=1,
)

if "sp500_close" in full_panel:
    fig.add_trace(
        go.Scatter(x=full_panel["date"], y=full_panel["sp500_close"], name="S&P 500", line=dict(color="blue")),
        row=2,
        col=1,
    )

if "usdt_supply" in full_panel:
    fig.add_trace(
        go.Scatter(
            x=full_panel["date"],
            y=full_panel["usdt_supply"] / 1e9,
            name="USDT Supply (B)",
            line=dict(color="green"),
        ),
        row=3,
        col=1,
        secondary_y=False,
    )
if "usdc_supply" in full_panel:
    fig.add_trace(
        go.Scatter(
            x=full_panel["date"],
            y=full_panel["usdc_supply"] / 1e9,
            name="USDC Supply (B)",
            line=dict(color="cyan"),
        ),
        row=3,
        col=1,
        secondary_y=True,
    )

if "fed_funds_rate" in full_panel:
    fig.add_trace(
        go.Scatter(
            x=full_panel["date"],
            y=full_panel["fed_funds_rate"],
            name="Fed Funds Rate",
            line=dict(color="red"),
        ),
        row=4,
        col=1,
    )
if "treasury_10y" in full_panel:
    fig.add_trace(
        go.Scatter(
            x=full_panel["date"],
            y=full_panel["treasury_10y"],
            name="10Y Treasury",
            line=dict(color="orange"),
        ),
        row=4,
        col=1,
    )

if "vix_close" in full_panel:
    fig.add_trace(
        go.Scatter(
            x=full_panel["date"],
            y=full_panel["vix_close"],
            name="VIX",
            line=dict(color="darkred"),
        ),
        row=5,
        col=1,
    )

fig.update_layout(height=1200, title_text="STX/BTC vs Macroeconomic Indicators (2-Year View)")
fig.update_xaxes(title_text="Date", row=5, col=1)

fig.show()

output_file = OUT_DIR / "stx_btc_macro_dashboard.html"
fig.write_html(output_file)
print(f"Dashboard saved to {output_file}")


In [ ]:

print("Testing Tether minting hypothesis with lead/lag analysis...")

if "usdt_daily_change" in full_panel and full_panel["usdt_daily_change"].notna().sum() > 0:
    lags = range(-30, 31)
    lag_correlations = []

    for lag in lags:
        shifted = full_panel["usdt_daily_change"].shift(lag)
        aligned = pd.concat([full_panel["btc_usd"], shifted], axis=1, join="inner").dropna()
        if len(aligned) < 2:
            corr = float("nan")
        else:
            corr = aligned["btc_usd"].corr(aligned["usdt_daily_change"])
        lag_correlations.append({"lag_days": lag, "correlation": corr})

    lag_df = pd.DataFrame(lag_correlations)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=lag_df["lag_days"],
            y=lag_df["correlation"],
            mode="lines+markers",
            name="Correlation",
            line=dict(color="green", width=2),
        )
    )
    fig.add_vline(x=0, line_dash="dash", line_color="red", annotation_text="No lag")
    fig.update_layout(
        title="USDT Supply Change vs BTC Price: Lead/Lag Correlation",
        xaxis_title="Lag (days) - Negative = Stablecoin leads BTC",
        yaxis_title="Correlation",
        height=400,
    )
    fig.show()

    if lag_df["correlation"].notna().any():
        peak_lag = lag_df.loc[lag_df["correlation"].idxmax()]
        print(f"\nPeak correlation: {peak_lag['correlation']:.4f} at lag {peak_lag['lag_days']} days")
        if peak_lag["lag_days"] < 0:
            print(f"  → USDT minting LEADS BTC price by {abs(peak_lag['lag_days'])} days")
        elif peak_lag["lag_days"] > 0:
            print(f"  → USDT minting LAGS BTC price by {peak_lag['lag_days']} days")
        else:
            print("  → USDT minting and BTC price move simultaneously")
    else:
        print("No valid correlations computed for lead/lag analysis.")
else:
    print("Insufficient stablecoin data for lead/lag analysis.")


In [ ]:

print("Analyzing STX/BTC behavior during high vs. low stablecoin minting...")

if "usdt_daily_change" in full_panel and full_panel["usdt_daily_change"].notna().sum() > 0:
    usdt_change_threshold_high = full_panel["usdt_daily_change"].quantile(0.75)
    usdt_change_threshold_low = full_panel["usdt_daily_change"].quantile(0.25)

    full_panel["stablecoin_regime"] = "Medium"
    full_panel.loc[
        full_panel["usdt_daily_change"] >= usdt_change_threshold_high, "stablecoin_regime"
    ] = "High Minting"
    full_panel.loc[
        full_panel["usdt_daily_change"] <= usdt_change_threshold_low, "stablecoin_regime"
    ] = "Low/Negative Minting"

    regime_stats = full_panel.groupby("stablecoin_regime").agg(
        {
            "stx_btc": ["mean", "std", "min", "max"],
            "btc_usd": ["mean", "std"],
            "sp500_pct_change": "mean",
        }
    )

    print("\nSTX/BTC statistics by stablecoin minting regime:")
    print(regime_stats)

    fig = go.Figure()
    for regime in ["High Minting", "Medium", "Low/Negative Minting"]:
        subset = full_panel[full_panel["stablecoin_regime"] == regime]
        fig.add_trace(
            go.Scatter(
                x=subset["date"],
                y=subset["stx_btc"],
                mode="markers",
                name=regime,
                marker=dict(size=4, opacity=0.6),
            )
        )

    fig.update_layout(
        title="STX/BTC Ratio by Stablecoin Minting Regime",
        xaxis_title="Date",
        yaxis_title="STX/BTC",
        height=500,
    )
    fig.show()
else:
    print("Insufficient stablecoin change data for regime analysis.")


In [ ]:

print("Exporting analysis results...")

OUT_DIR.mkdir(parents=True, exist_ok=True)

full_panel.to_parquet(OUT_DIR / "stx_btc_macro_panel.parquet")
full_panel.to_csv(OUT_DIR / "stx_btc_macro_panel.csv", index=False)

if 'correlations' in locals():
    correlations.to_csv(OUT_DIR / "stx_btc_correlations.csv")
if 'lag_df' in locals():
    lag_df.to_csv(OUT_DIR / "usdt_btc_leadlag_analysis.csv", index=False)
if 'regime_stats' in locals():
    regime_stats.to_csv(OUT_DIR / "stablecoin_regime_statistics.csv")

print("Analysis complete! Outputs saved to out/ directory:")
for filename in [
    "stx_btc_macro_panel.parquet",
    "stx_btc_macro_panel.csv",
    "stx_btc_correlations.csv",
    "usdt_btc_leadlag_analysis.csv",
    "stablecoin_regime_statistics.csv",
    "stx_btc_macro_dashboard.html",
]:
    print(f"  - {filename}")
